In [1]:
# from mobile_net import get_model
import librosa
import tensorflow as tf
import glob
import numpy as np
import random
import wave
import os
import pickle
# import noisereduce as nr
import requests

2024-10-30 23:09:21.137029: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 23:09:21.150487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-30 23:09:21.166288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-30 23:09:21.171050: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-30 23:09:21.183434: I tensorflow/core/platform/cpu_feature_guar

In [2]:
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_logical_devices('GPU')

2024-10-30 23:09:22.826704: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43629 MB memory:  -> device: 0, name: NVIDIA L40S, pci bus id: 0000:8b:00.0, compute capability: 8.9


[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
noise_path = '/shareddrive/working/data_code/data/neg_data/_background_noise_/chunks/2-5s_chunks/*'
environment_path = '/shareddrive/working/data_code/data/neg_data/envornment/chunks/2-5s_chunks/*'
word_path = '/shareddrive/working/data_code/data/neg_data/spcmd/all_words/original/*'
# word_path = '/shareddrive/working/data_code/data/neg_data/spcmd/all_words/2-5s_chunks/*'
recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks/*'
aug_recording_path = '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/*'

In [5]:
word_files = glob.glob(word_path)
environment_files = glob.glob(environment_path)
noise_files = glob.glob(noise_path)
recording_files = glob.glob(recording_path)
aug_recording_files = glob.glob(aug_recording_path)
len(environment_files),len(noise_files),len(word_files),len(recording_files),len(aug_recording_files)

(515, 264, 64721, 8649, 16636)

In [6]:
# def get_duration(audio_path):
#     with wave.open(audio_path, 'rb') as wav_file:
#         sample_rate = wav_file.getframerate()
#         num_frames = wav_file.getnframes()
#         duration = num_frames / sample_rate
#     return duration

In [7]:
limit = 1.5
keyword_path = '/shareddrive/working/data_code/data/adele/augmented/*'
keyword_folders = glob.glob(keyword_path)
keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*')]
# keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]
# keyword_path = '/shareddrive/working/data_code/data/hilfe_hilfe/augmented/*'
# keyword_folders = glob.glob(keyword_path)
# keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*')]
# keyword_files = [j for i in keyword_folders for j in glob.glob(f'{i}/*') if get_duration(j) <= limit]

In [8]:
len(keyword_files)

14250

In [9]:
# DEEPGRAM_API_KEY = '4207b8a639744fbdbe634616684bf7d67c8791e2'
# url = 'https://api.deepgram.com/v1/listen'
# headers = {
#     'Authorization': f'Token {DEEPGRAM_API_KEY}',
#     'Content-Type': 'audio/wav'
# }
# params = {
#     'language': 'de',  # Specify the language as German
#     'tier': 'enhanced',  # Use the enhanced tier for better accuracy
#     'model': 'general'  # Specify the model
# }
# ad_files = list()
# for i in adele_files:
#     # Read the audio file
#     with open(i, 'rb') as audio_file:
#         response = requests.post(url, headers=headers, params=params, data=audio_file)

#     if response.status_code == 200:
#         if 'adele' in response.json()['results']['channels'][0]['alternatives'][0]['transcript']:
#             ad_files.append(i)
#     else:
#         print("Error:", response.json())
# len(ad_files)

In [10]:
no_of_files = 1109+660
file_path_and_labels = list()
# file_path_and_labels.extend([(i,1) for i in random.sample(adele_files,no_of_files)])
file_path_and_labels.extend([(i,1) for i in keyword_files])
# avg_files = (len(adele_files) + len(hilfe_files)) // 2
avg_files = int(len(keyword_files)/8)
file_path_and_labels.extend([(i,0) for i in random.sample(aug_recording_files,int(avg_files*1.8))])
file_path_and_labels.extend([(i,0) for i in recording_files])
# file_path_and_labels.extend([(i,0) for i in aug_recording_files])
file_path_and_labels.extend([(i,0) for i in random.sample(word_files,avg_files)])
file_path_and_labels.extend([(i,0) for i in noise_files])
file_path_and_labels.extend([(i,0) for i in environment_files])
random.shuffle(file_path_and_labels)
# avg_files
len(file_path_and_labels),len(keyword_files),len(file_path_and_labels)-len(keyword_files),

(28664, 14250, 14414)

In [11]:
aug_recording_files[:20]

['/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_00971196588c18.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_0158312d4f0770.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_01871f09884cd9.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_0196c8eea5ce69.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_01bba8d5689f14.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_01c43346f4810d.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_01e05a3f58c559.wav',
 '/shareddrive/working/data_code/data/neg_data/internet_recordings/chunks/2-5s_chunks/2-5s_chunks_aug/bc_01f0eacb0ea01

In [12]:
sr = 16000
max_seconds = 1#2.5
pad_or_trunc = lambda a,i : a[0:i] if len(a) > i else a if len(a) == i else np.pad(a,(0, (i-len(a))))

def process_data(y,sr,max_seconds):
    y = pad_or_trunc(y,int(sr*max_seconds))
    features = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=1024)
    return features
    
def get_processed_data(audio_file):
    y,_ = librosa.load(audio_file,sr=sr)
    # y = nr.reduce_noise(y=y, sr=sr,n_fft=1024)
    # y[np.isnan(y)] = 0
    features = process_data(y,sr,max_seconds)
    return features
try:
    with open('f_and_ls.pickle','rb') as f:
        features_and_labels = pickle.load(f)
except:
    features_and_labels = list()
    for i,j in file_path_and_labels:
        try:
            features = get_processed_data(i)
            features_and_labels.append((features,j))
        except Exception as e: 
            print(i)
            print(e)
        # features = get_processed_data(i)
        # features_and_labels.append(features,j)
    # with open('f_and_l.pickle','wb') as f:
    #     pickle.dump(features_and_labels,f)
finally:
    print(features_and_labels[0][0].shape)

(128, 32)


In [13]:


class DataSequenceRaw(tf.keras.utils.Sequence):

    def __init__(self,data,batch_size):
        self.data_size = len(data)
        audios,labels = zip(*data)
        self.X = np.array(audios)
        self.Y = np.array(labels)
        self.batch_size = batch_size

        
    def __len__(self):
        return int(np.ceil(self.data_size / self.batch_size))

    def __getitem__(self,idx):
        s = idx * self.batch_size
        e = (idx + 1) * self.batch_size
        X = self.X[s:e]
        Y = self.Y[s:e]
        return X,Y


In [14]:
def _get_train_val_size(total_examples,train_percent=70):
    
    train_ratio = round(train_percent)/100
    remaining_percent = 100 - train_percent
    test_percent = round(remaining_percent/3)*2
    val_percent = remaining_percent - test_percent
    val_ratio = val_percent/100
    train_size = round(total_examples * train_ratio)
    val_size = round(total_examples * val_ratio)
    return train_size,val_size

def get_data_raw(data,train_percent=70,batch_size=32):

    train_size,val_size = _get_train_val_size(len(data),train_percent)

    train_examples = data[:train_size]
    val_examples = data[train_size:train_size+val_size]
    test_examples = data[train_size+val_size:]

    train = DataSequenceRaw(train_examples,batch_size=batch_size)
    test = DataSequenceRaw(test_examples,batch_size=1)
    val = DataSequenceRaw(val_examples,batch_size=batch_size)

    return train,test,val

train,test,val = get_data_raw(features_and_labels,train_percent=80)

In [15]:
shape = train[0][0][0].shape
input_shape = [*shape,1]

In [16]:
def get_model(
        input_shape,
        output_neurons=1,
        output_activation='sigmoid',
        loss=tf.keras.losses.binary_crossentropy,
        lr=0.0001
):
    _input = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(512,kernel_size=3,padding='valid',activation='relu')(_input)
    x = tf.keras.layers.Conv2D(256,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Conv2D(128,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv2D(128,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Conv2D(64,kernel_size=3,padding='valid',activation='relu')(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    x = tf.keras.layers.Dense(1024,activation='relu')(x)
    x = tf.keras.layers.Dense(10,activation='relu')(x)
    outputs = tf.keras.layers.Dense(output_neurons,activation=output_activation,kernel_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
    model = tf.keras.Model(inputs=_input,outputs=outputs)

    model.compile(
        loss=loss,
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        metrics=['accuracy'],
    )

    return model

model = get_model(
        input_shape=input_shape,
        lr=0.001
)



In [17]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',factor=0.1,patience=3,mode='max')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=5,mode='max',restore_best_weights=True,start_from_epoch=10)
with tf.device('/gpu'):
    history = model.fit(train,epochs=100,validation_data=val,verbose=1,callbacks=[reduce_lr,early_stopping])

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1730330813.516126    3956 service.cc:146] XLA service 0x7fa168004660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730330813.516160    3956 service.cc:154]   StreamExecutor device (0): NVIDIA L40S, Compute Capability 8.9
2024-10-30 23:26:53.572778: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-30 23:26:53.913966: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
2024-10-30 23:26:54.946251: I external

 10/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.5307 - loss: 0.8412

I0000 00:00:1730330820.055048    3956 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


559/717 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.8484 - loss: 0.4508

2024-10-30 23:27:11.676394: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2069', 232 bytes spill stores, 232 bytes spill loads



716/717 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8596 - loss: 0.4197

2024-10-30 23:27:19.449484: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 132 bytes spill stores, 132 bytes spill loads

2024-10-30 23:27:21.986207: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 32 bytes spill stores, 32 bytes spill loads

2024-10-30 23:27:22.132793: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 200 bytes spill stores, 200 bytes spill loads

2024-10-30 23:27:22.364270: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_102', 12 bytes spill stores, 12 bytes spill loads

2024-10-30 23:27:22.591979: I external/local_xla/xla/stream_

717/717 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - accuracy: 0.8597 - loss: 0.4194 - val_accuracy: 0.9529 - val_loss: 0.1478 - learning_rate: 0.0010
Epoch 2/100
717/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9557 - loss: 0.1442 - val_accuracy: 0.9727 - val_loss: 0.0858 - learning_rate: 0.0010
Epoch 3/100
717/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9530 - loss: 0.1670 - val_accuracy: 0.9802 - val_loss: 0.0781 - learning_rate: 0.0010
Epoch 4/100
717/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9763 - loss: 0.0800 - val_accuracy: 0.9907 - val_loss: 0.0294 - learning_rate: 0.0010
Epoch 5/100
717/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9816 - loss: 0.0671 - val_accuracy: 0.9936 - val_loss: 0.0228 - learning_rate: 0.0010
Epoch 6/100
717/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9865 - loss: 0.0468 - val_accuracy: 0.9913 - val_loss: 0.0219 - learning_rate: 0.0010
Epoch 7/100
717/717 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.9884 - loss: 

In [18]:
model.evaluate(test)

4013/4013 ━━━━━━━━━━━━━━━━━━━━ 5s 994us/step - accuracy: 0.9951 - loss: 0.0183


[0.022720249369740486, 0.9937702417373657]

In [19]:
folder_path = '/shareddrive/working/model_code/models/custom_model_4/trail_7'
os.makedirs(folder_path,exist_ok=True)
model_path = f'{folder_path}/16k_melspec-nfft-1024_a_cnn_dense_model.keras'
model.save(model_path)